# Step 1: Preprocess the Data

## Importing Dependencies, Loading data

In [24]:
!pip install keras-tuner

In [25]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as skl
import tensorflow as tf
import keras_tuner as kt




#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from keras_tuner import Hyperband

### What variable(s) are the target(s) for the model?

The target variable for our Neural Network model is "IS_SUCCESSFUL," which is a binary variable. When "IS_SUCCESSFUL" equals 1, it indicates that the applicant was successful after being funded by Alphabet Soup. Conversely, when "IS_SUCCESSFUL" equals 0, it signifies that the applicant was not successful.


### What variable(s) are the feature(s) for the model?
We considered all other variable except(EIN, NAME) as the features and input of our model.

## Preprocessing

In [27]:
print(application_df.columns)

Index(['EIN', 'NAME', 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',
       'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT',
       'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')


In [28]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

application_df = application_df.reset_index(drop=True)
application_df = application_df.drop(columns=['EIN','NAME'])

In [29]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [30]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [31]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

app_df = pd.DataFrame(application_df['APPLICATION_TYPE'].value_counts())
application_types_to_replace = []


cutoff_value=500

for index, row in app_df.iterrows():
    if row['APPLICATION_TYPE'] < cutoff_value:
        application_types_to_replace.append(index)

print(application_types_to_replace)


['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']


In [32]:

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [33]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [34]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_classification_gt_1=application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts()>1]
application_classification_gt_1

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [35]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

app_class_df=pd.DataFrame(application_df['CLASSIFICATION'].value_counts())

cutoff=1000

classifications_to_replace=[]
for index, row in app_class_df.iterrows():
    if row['CLASSIFICATION']<cutoff:
        classifications_to_replace.append(index)

print(classifications_to_replace)

['C7000', 'C1700', 'C4000', 'C5000', 'C1270', 'C2700', 'C2800', 'C7100', 'C1300', 'C1280', 'C1230', 'C1400', 'C7200', 'C2300', 'C1240', 'C8000', 'C7120', 'C1500', 'C1800', 'C6000', 'C1250', 'C8200', 'C1238', 'C1278', 'C1235', 'C1237', 'C7210', 'C2400', 'C1720', 'C4100', 'C1257', 'C1600', 'C1260', 'C2710', 'C0', 'C3200', 'C1234', 'C1246', 'C1267', 'C1256', 'C2190', 'C4200', 'C2600', 'C5200', 'C1370', 'C1248', 'C6100', 'C1820', 'C1900', 'C1236', 'C3700', 'C2570', 'C1580', 'C1245', 'C2500', 'C1570', 'C1283', 'C2380', 'C1732', 'C1728', 'C2170', 'C4120', 'C8210', 'C2561', 'C4500', 'C2150']


In [36]:

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [37]:
application_df.head(4)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1


In [38]:
application_df_cat=application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_df_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [39]:
# Convert categorical data to numeric with `pd.get_dummies`

application_df_got_dummies=pd.get_dummies(application_df, columns=application_df_cat)
application_df_got_dummies


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [40]:
application_df.columns

Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS',
       'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [41]:
application_df_got_dummies.columns

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_Other', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare',
       'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M

In [42]:
len(application_df_got_dummies.columns)

44

In [43]:
# Split our preprocessed data into our features and target arrays

y=application_df_got_dummies.IS_SUCCESSFUL.values
X=application_df_got_dummies.drop(columns="IS_SUCCESSFUL").values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)


In [44]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Step 2: Compile, Train, and Evaluate the Model

## Define the model and Determine the layers

**Create method that create a new Sequential model with Hyperparameters options**

In [45]:

def create_model(hp):
    model = Sequential()

    # Hyperparameters to tune
    activation_choice = hp.Choice('activation', values=['relu', 'elu', 'selu', 'tanh'])
    dropout_rate = hp.Float('dropout', 0, 0.5, step=0.1)
    l2_rate = hp.Float('l2_rate', 1e-5, 1e-2, sampling='log')

    # Input layer
    model.add(Dense(units=hp.Int('units_input',
                                 min_value=32,
                                 max_value=512,
                                 step=32),
                    activation=activation_choice,
                    input_dim=43,
                    kernel_regularizer=l2(l2_rate)))
    model.add(Dropout(rate=dropout_rate))

    # Hidden layers
    for i in range(hp.Int('num_layers', 1, 8)):
        model.add(Dense(units=hp.Int(f'units_layer_{i}',
                                     min_value=16,
                                     max_value=256,
                                     step=16),
                        activation=activation_choice,
                        kernel_regularizer=l2(l2_rate)))
        model.add(Dropout(rate=dropout_rate))

    # Output layer
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model



In [46]:
# Instantiate the tuner
tuner = Hyperband(
    create_model,
    objective='val_accuracy',
    max_epochs=20,
    directory='my_dir',
    project_name='hyperband_tuning'
)

# Early stopping callback
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Execute the search
tuner.search(X_train_scaled, y_train,
             epochs=50,
             validation_split=0.2,
             callbacks=[stop_early],
             verbose=2)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the best hyperparameters
model = tuner.hypermodel.build(best_hps)

# Train the best model
history = model.fit(X_train_scaled, y_train,
                    epochs=50,
                    validation_split=0.2,
                    callbacks=[stop_early],
                    verbose=2)



Trial 26 Complete [00h 00m 33s]
val_accuracy: 0.7259474992752075

Best val_accuracy So Far: 0.7284742593765259
Total elapsed time: 00h 15m 06s
Epoch 1/50
644/644 - 5s - loss: 0.6046 - accuracy: 0.7016 - val_loss: 0.5770 - val_accuracy: 0.7271 - 5s/epoch - 8ms/step
Epoch 2/50
644/644 - 2s - loss: 0.5785 - accuracy: 0.7265 - val_loss: 0.5729 - val_accuracy: 0.7224 - 2s/epoch - 4ms/step
Epoch 3/50
644/644 - 2s - loss: 0.5707 - accuracy: 0.7294 - val_loss: 0.5705 - val_accuracy: 0.7230 - 2s/epoch - 3ms/step
Epoch 4/50
644/644 - 2s - loss: 0.5701 - accuracy: 0.7269 - val_loss: 0.5767 - val_accuracy: 0.7252 - 2s/epoch - 4ms/step
Epoch 5/50
644/644 - 3s - loss: 0.5684 - accuracy: 0.7313 - val_loss: 0.5690 - val_accuracy: 0.7244 - 3s/epoch - 5ms/step
Epoch 6/50
644/644 - 2s - loss: 0.5670 - accuracy: 0.7290 - val_loss: 0.5723 - val_accuracy: 0.7190 - 2s/epoch - 3ms/step
Epoch 7/50
644/644 - 2s - loss: 0.5637 - accuracy: 0.7289 - val_loss: 0.5791 - val_accuracy: 0.7211 - 2s/epoch - 3ms/step
Epo

In [47]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

268/268 - 0s - loss: 0.5693 - accuracy: 0.7203 - 403ms/epoch - 2ms/step
Test Loss: 0.5692663192749023, Test Accuracy: 0.7203498482704163


In [48]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 224)               9856      
                                                                 
 dropout_8 (Dropout)         (None, 224)               0         
                                                                 
 dense_10 (Dense)            (None, 48)                10800     
                                                                 
 dropout_9 (Dropout)         (None, 48)                0         
                                                                 
 dense_11 (Dense)            (None, 16)                784       
                                                                 
 dropout_10 (Dropout)        (None, 16)                0         
                                                                 
 dense_12 (Dense)            (None, 192)              

## Export the model to HDF5

In [49]:
# Export our model to HDF5 file

# Import necessary libraries
import h5py

# Save the model to HDF5 file
model.save("AlphabetSoupCharity_Optimisation_3.h5")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [50]:
pwd

'/content'